In [1]:
import pandas as pd
import scanpy as sc
import h5py
import CococoNet_reader
import numpy as np
import anndata
import pickle
import tqdm
import Go_annotations
import seaborn as sns
import matplotlib.pyplot as plt
sc.settings.verbosity = 3  
sc.set_figure_params(facecolor = 'white', figsize = (10,8))

In [2]:
import numpy as np
import pandas as pd
from scipy import stats, sparse
import bottleneck


def run_egad(go, nw, **kwargs):
    """EGAD running function
    
    Wrapper to lower level functions for EGAD

    EGAD measures modularity of gene lists in co-expression networks. 

    This was translated from the MATLAB version, which does tiled Cross Validation
    
    The useful kwargs are:
    int - nFold : Number of CV folds to do, default is 3, 
    int - {min,max}_count : limits for number of terms in each gene list, these are exclusive values


    Arguments:
        go {pd.DataFrame} -- dataframe of genes x terms of values [0,1], where 1 is included in gene lists
        nw {pd.DataFrame} -- dataframe of co-expression network, genes x genes
        **kwargs 
    
    Returns:
        pd.DataFrame -- dataframe of terms x metrics where the metrics are 
        ['AUC', 'AVG_NODE_DEGREE', 'DEGREE_NULL_AUC', 'P_Value']
    """
    assert nw.shape[0] == nw.shape[1] , 'Network is not square'
    assert np.all(nw.index == nw.columns) , 'Network index and columns are not in the same order'
    nw_mask = nw.isna().sum(axis=1) != nw.shape[1]
    nw = nw.loc[nw_mask, nw_mask].astype(float)
    np.fill_diagonal(nw.values, 1)
    return _runNV(go, nw, **kwargs)


def _runNV(go, nw, nFold=3, min_count=20, max_count=1000):

    #Make sure genes are same in go and nw
    genes_intersect = go.index.intersection(nw.index)

    go = go.loc[genes_intersect, :]
    nw = nw.loc[genes_intersect, genes_intersect]

    #Make sure there aren't duplicates
    duplicates = nw.index.duplicated(keep='first')
    nw = nw.loc[~duplicates, ~duplicates]

    go = go.loc[:, (go.sum(axis=0) > min_count) & (go.sum(axis=0) < max_count)]
    go = go.loc[~go.index.duplicated(keep='first'), :]

    roc = _new_egad(go.values, nw.values, nFold)

    col_names = ['AUC', 'AVG_NODE_DEGREE', 'DEGREE_NULL_AUC', 'P_Value']
    #Put output in dataframe
    return pd.DataFrame(dict(zip(col_names, roc)), index=go.columns)


def _new_egad(go, nw, nFold):

    #Build Cross validated Positive
    x, y = np.where(go)
    cvgo = {}
    for i in np.arange(nFold):
        a = x[i::nFold]
        b = y[i::nFold]
        dat = np.ones_like(a)
        mask = sparse.coo_matrix((dat, (a, b)), shape=go.shape)
        cvgo[i] = go - mask.toarray()
        
    CVgo = np.concatenate(list(cvgo.values()), axis=1)

    sumin = np.matmul(nw.T, CVgo)

    degree = np.sum(nw, axis=0)

    predicts = sumin / degree[:, None]

    np.place(predicts, CVgo > 0, np.nan)

    #Calculate ranks of positives
    rank_abs = lambda x: stats.rankdata(np.abs(x))
    predicts2 = np.apply_along_axis(rank_abs, 0, predicts)

    #Masking Nans that were ranked (how tiedrank works in matlab)
    predicts2[np.isnan(predicts)] = np.nan

    filtering = np.tile(go, nFold)

    #negatives :filtering == 0
    #Sets Ranks of negatives to 0
    np.place(predicts2, filtering == 0, 0)

    #Sum of ranks for each prediction
    p = bottleneck.nansum(predicts2, axis=0)

    #Number of predictions
    #Number of 1's masked for each GO term for each CV
    n_p = np.sum(filtering, axis=0) - np.sum(CVgo, axis=0)

    #Number of negatives
    #Number of GO terms - number of postiive
    n_n = filtering.shape[0] - np.sum(filtering, axis=0)

    roc = (p / n_p - (n_p + 1) / 2) / n_n
    U = roc * n_p * n_n
    Z = (np.abs(U - (n_p * n_n / 2))) / np.sqrt(n_p * n_n *
                                                (n_p + n_n + 1) / 12)
    roc = roc.reshape(nFold, go.shape[1])
    Z = Z.reshape(nFold, go.shape[1])
    #Stouffer Z method
    Z = bottleneck.nansum(Z, axis=0) / np.sqrt(nFold)
    #Calc ROC of Neighbor Voting
    roc = bottleneck.nanmean(roc, axis=0)
    P = stats.norm.sf(Z)

    #Average degree for nodes in each go term
    avg_degree = degree.dot(go) / np.sum(go, axis=0)

    #Calc null auc for degree
    ranks = np.tile(stats.rankdata(degree), (go.shape[1], 1)).T

    np.place(ranks, go == 0, 0)

    n_p = bottleneck.nansum(go, axis=0)
    nn = go.shape[0] - n_p
    p = bottleneck.nansum(ranks, axis=0)

    roc_null = (p / n_p - ((n_p + 1) / 2)) / nn

    return roc, avg_degree, roc_null, P


In [3]:
single_cell_h5py = h5py.File('/data/passala/Data_from_CoCoCoNet/single_cell_data/Ara_data.hdf5','r')
list(single_cell_h5py.keys())
col_data = single_cell_h5py['coldata']
row_data = single_cell_h5py['rowdata']
embedding_data = single_cell_h5py['embedding']
normalized_counts = single_cell_h5py['normalized_counts']

row_data_decoded = []

for gene_name in row_data:
    row_data_decoded.append(gene_name[0].decode())

cell_type_number =[]
study_number = []
study_id = []
batch_cluster = []
meta_cluster = []
umap_coordinates = []

for cell_identity in col_data:
    cell_type_number.append(cell_identity[0])  
    study_number.append(cell_identity[1])
    study_id.append(cell_identity[2])
    batch_cluster.append(cell_identity[3])
    meta_cluster.append(cell_identity[4])

barcode_for_each_cell = [] 
for barcode in embedding_data:
    barcode_for_each_cell.append(barcode[2])
    current_umap_coordinates =[barcode[1],barcode[0]]
    umap_coordinates.append(current_umap_coordinates)
umap_coordinates = np.array(umap_coordinates)


In [4]:
obs_arabidop = pd.DataFrame(index = barcode_for_each_cell, data = list(zip(cell_type_number,study_number,batch_cluster, meta_cluster)), columns = ['Cell Type','Study Number','Batch Cluster','Meta Cluster'] )

vars_arabidop = pd.DataFrame(index = row_data_decoded)
single_cell_arabidopsis_root_4_datasets = anndata.AnnData(X = normalized_counts[:],obs = obs_arabidop, var = vars_arabidop)
single_cell_arabidopsis_root_4_datasets.obsm['X_umap'] = umap_coordinates
umap_df = pd.DataFrame(data = single_cell_arabidopsis_root_4_datasets.obsm['X_umap'], columns = ['Axis 1','Axis 2'], index = barcode_for_each_cell)
bad_values = umap_df.sort_values(by = 'Axis 1', ascending = False).head(6).index
single_cell_arabidopsis_root_4_datasets.obs.loc[bad_values]
good_obs = single_cell_arabidopsis_root_4_datasets.obs.loc[~single_cell_arabidopsis_root_4_datasets.obs.index.isin(bad_values)]
single_cell_arabidopsis_root_4_datasets  = single_cell_arabidopsis_root_4_datasets[good_obs.index,:]

/tmp/ipykernel_1100828/2112258600.py:4: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  single_cell_arabidopsis_root_4_datasets = anndata.AnnData(X = normalized_counts[:],obs = obs_arabidop, var = vars_arabidop)


In [5]:
single_cell_arabidopsis_root_4_datasets.obs['Meta Cluster String'] =single_cell_arabidopsis_root_4_datasets.obs['Meta Cluster'].astype(str)

/tmp/ipykernel_1100828/1255668264.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  single_cell_arabidopsis_root_4_datasets.obs['Meta Cluster String'] =single_cell_arabidopsis_root_4_datasets.obs['Meta Cluster'].astype(str)


In [6]:
sc.pp.filter_cells(single_cell_arabidopsis_root_4_datasets, min_genes=300)
sc.pp.filter_genes(single_cell_arabidopsis_root_4_datasets, min_cells=10)

filtered out 1196 genes that are detected in less than 10 cells


In [7]:
sc.pp.highly_variable_genes(single_cell_arabidopsis_root_4_datasets, min_mean=0.125, max_mean=4, min_disp=0.5)


extracting highly variable genes


In [ ]:
sc.pl.highly_variable_genes(single_cell_arabidopsis_root_4_datasets, )


In [ ]:
#sc.pl.highly_variable_genes(single_cell_arabidopsis_root_4_datasets)


In [ ]:
sc.tl.pca(single_cell_arabidopsis_root_4_datasets, svd_solver='arpack', random_state=303)


In [ ]:
sc.pl.pca_variance_ratio(single_cell_arabidopsis_root_4_datasets, log=True)


In [ ]:
sc.pp.neighbors(single_cell_arabidopsis_root_4_datasets, n_neighbors=12, n_pcs=50)


In [ ]:
sc.tl.umap(single_cell_arabidopsis_root_4_datasets, random_state = 233)

In [ ]:
sc.pl.umap(single_cell_arabidopsis_root_4_datasets,color = 'Meta Cluster String', s = 20)

In [ ]:
sc.tl.leiden(single_cell_arabidopsis_root_4_datasets,resolution = 15, random_state = 325)

In [ ]:
single_cell_arabidopsis_root_4_datasets.obs['leiden'].value_counts().tail(20)

In [ ]:
single_cell_arabidopsis_root_4_datasets

In [ ]:
single_cell_arabidopsis_root_4_datasets = single_cell_arabidopsis_root_4_datasets[:,single_cell_arabidopsis_root_4_datasets.var['means']>0]

In [ ]:
psuedobulk_df = pd.DataFrame(index = single_cell_arabidopsis_root_4_datasets.var_names)
all_samples = list(single_cell_arabidopsis_root_4_datasets.obs.leiden.unique()) # Generate list of all combinations of the above ## Make a base dataframe index we will add stuff on to later
psuedobulk_df

In [ ]:
len(all_samples)

In [ ]:
for batch_type in all_samples:

    ## Read in the Names so our code is easy to understand
    current_cluster = batch_type

    ## Calculate the Psuedobulked mean
    cells_matching_batch_and_cluster = single_cell_arabidopsis_root_4_datasets[single_cell_arabidopsis_root_4_datasets.obs['leiden'] == current_cluster ]
    mean_of_genes = cells_matching_batch_and_cluster.X.mean(axis = 0).tolist()


    name_of_combo = current_cluster
    psuedobulk_df[name_of_combo] = mean_of_genes

In [ ]:
psuedobulk_df

In [ ]:
exp_data = psuedobulk_df

In [ ]:
import numpy as np
import scipy.stats as sci

rank_test_py_exp = sci.rankdata(exp_data, method = 'average', axis = 1)                #Row ranks
rank_test_py_exp = rank_test_py_exp - rank_test_py_exp.mean(axis = 1)[1]               #Center each gene, subtract mean rank
rank_test_py_exp_2 = np.square(rank_test_py_exp)                                       #Square
rank_test_py_exp = rank_test_py_exp /np.sqrt(rank_test_py_exp_2.sum(axis = 1))[:,None] #divide by sqrt(rowSums)
cr_python = np.dot(rank_test_py_exp, rank_test_py_exp.T)                               # Get correlations

In [ ]:
cr_python

In [ ]:
corr_results = pd.DataFrame(columns = psuedobulk_df.index, index = psuedobulk_df.index, data = cr_python)
corr_results

In [ ]:
arabi_net = CococoNet_reader.read_cococonet('arabidopsis')
arabi_net

In [ ]:
arabi_net_col_trimmed = arabi_net[arabi_net.columns.intersection(corr_results.index)]
arabi_net_both_trimmed = arabi_net_col_trimmed.loc[arabi_net_col_trimmed.index.isin(corr_results.index)]

In [ ]:
arabi_net_both_trimmed

In [ ]:
corr_results

In [ ]:
corr_results_col_trimmed = corr_results[corr_results.columns.intersection(arabi_net_both_trimmed.index)]
corr_results_both_trimmed = corr_results_col_trimmed.loc[corr_results_col_trimmed.index.isin(arabi_net_both_trimmed.index)]

In [ ]:
corr_results_both_trimmed

In [ ]:
corr_results.reindex()

In [ ]:
corr_results_both_trimmed = corr_results_both_trimmed.clip(lower=0)


In [ ]:
corr_results_both_trimmed = corr_results_both_trimmed[arabi_net_both_trimmed.columns]
corr_results_both_trimmed = corr_results_both_trimmed.T
corr_results_both_trimmed = corr_results_both_trimmed[arabi_net_both_trimmed.columns]
corr_results_both_trimmed

In [ ]:
# %%script false --no-raise-error
# sns.clustermap(corr_results_both_trimmed)

In [ ]:
# %%script false --no-raise-error
# sns.clustermap(arabi_net_both_trimmed)

In [ ]:
arabi_net_both_trimmed

In [ ]:
## Trim cococonets to match


arab_cococonet_1_set_to_zero = arabi_net_both_trimmed.replace(1,0)
corr_results_both_trimmed = corr_results_both_trimmed.replace(1,0)


top_10_arab_genes = np.array(
    [arab_cococonet_1_set_to_zero[c].nlargest(10).index.values for c in arab_cococonet_1_set_to_zero]
)  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
top_10_arab_genes_dataframe = pd.DataFrame(
    data=top_10_arab_genes,
    index=arab_cococonet_1_set_to_zero.index,
    columns=[
        "One",
        "Two",
        "Three",
        "Four",
        "Five",
        "Six",
        "Seven",
        "Eight",
        "Nine",
        "Ten",
    ],
)


tidy_top_10 = top_10_arab_genes_dataframe.melt(ignore_index= False)

zipped_pairs = zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))

binary_masked_cococonet = pd.DataFrame(data = 0, columns = arab_cococonet_1_set_to_zero.columns, index = arab_cococonet_1_set_to_zero.index)
#binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
#binary_masked_cococonet.sum(axis =0)
for row,column in zipped_pairs:
    binary_masked_cococonet.at[row,column] = 1

binary_masked_cococonet

ranked_columns_cococonet = corr_results_both_trimmed.rank()
dot_product_cococonet = binary_masked_cococonet.dot(ranked_columns_cococonet)
subtract_minimum = dot_product_cococonet-65 # This is 11+10+9+8+7+6+5+4+3+2
subtract_minimum
function_conservations_scores = subtract_minimum/(subtract_minimum.max().max() - subtract_minimum.min().min())
function_conservations_scores

In [ ]:
#ranked_function_conservation_scores = function_conservations_scores.rank()

In [ ]:
# normalized_ranked_function_conservation_scores = ranked_function_conservation_scores/19211
# normalized_ranked_function_conservation_scores

In [ ]:
np.diag(function_conservations_scores)[0:100]

In [ ]:
np.median(np.diag(function_conservations_scores))

In [ ]:
 ax = sns.histplot(np.diag(function_conservations_scores))
 ax.grid(False)